In [ ]:
#n번정점을 업데이트 하려면 오일러투어 정점 리스트의 n번정점의 s~e까지 그 값으로 업데이트 해야한다.
import io, os, sys
sys.setrecursionlimit(10**5)
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class LazySegmentTree:
  def __init__(self, data, default=0, func=max):
    """initialize the lazy segment tree with data"""
    self._default = default
    self._func = func

    self._len = len(data)
    self._size = _size = 1 << (self._len - 1).bit_length()
    self._lazy = [0] * (2 * _size)

    self.data = [default] * (2 * _size)
    self.data[_size:_size + self._len] = data
    for i in reversed(range(_size)):
      self.data[i] = func(self.data[i + i], self.data[i + i + 1])

  def _push(self, idx):
    """push query on idx to its children"""
    # Let the children know of the queries
    q, self._lazy[idx] = self._lazy[idx], 0

    self._lazy[2 * idx] += q
    self._lazy[2 * idx + 1] += q
    self.data[2 * idx] += q
    self.data[2 * idx + 1] += q

  def _update(self, idx):
    """updates the node idx to know of all queries applied to it via its ancestors"""
    for i in reversed(range(1, idx.bit_length())):
      self._push(idx >> i)

  def _build(self, idx):
    """make the changes to idx be known to its ancestors"""
    idx >>= 1
    while idx:
      self.data[idx] = self._func(self.data[2 * idx], self.data[2 * idx + 1]) + self._lazy[idx]
      idx >>= 1

  def add(self, start, stop, value):
    """lazily add value to [start, stop)"""
    start = start_copy = start + self._size
    stop = stop_copy = stop + self._size
    while start < stop:
      if start & 1:
        self._lazy[start] += value
        self.data[start] += value
        start += 1
      if stop & 1:
        stop -= 1
        self._lazy[stop] += value
        self.data[stop] += value
      start >>= 1
      stop >>= 1

    # Tell all nodes above of the updated area of the updates
    self._build(start_copy)
    self._build(stop_copy - 1)

  def query(self, start, stop, default=0):
    """func of data[start, stop)"""
    start += self._size
    stop += self._size

    # Apply all the lazily stored queries
    self._update(start)
    self._update(stop - 1)

    res = default
    while start < stop:
      if start & 1:
        res = self._func(res, self.data[start])
        start += 1
      if stop & 1:
        stop -= 1
        res = self._func(res, self.data[stop])
      start >>= 1
      stop >>= 1
    return res

def sol() :
  cnt = 0
  def dfs(x, p) :
    nonlocal cnt
    cnt += 1
    S[x] = cnt
    for nx in G[x] :
      if nx == p : continue
      dfs(nx, x)
    E[x] = cnt

  N, M = map(int, input().split())
  G = [[] for _ in range(N+1)]
  for i, p in enumerate(map(int, input().split())) :
    if p == -1 : continue
    G[p].append(i+1)
  
  S = [0] * (N+1) #i번 정점에 방문한 시점
  E = [0] * (N+1) #i번 정점을 빠져나온 시점

  dfs(1, -1) #승범이는 항상 1번
  LST = LazySegmentTree([0] * (N+1), 0, lambda x, y : x+y)
  ans = []
  for _ in range(M) :
    q, *l = map(int, input().split())
    if q == 1 :
      x, w = l
      LST.add(S[x], E[x]+1, w)
    elif q == 2 :
      x = l[0]
      ans.append(LST.query(S[x], S[x]+1))
  
  sys.stdout.write('\n'.join(map(str, ans)))
  
sol()

- iterative DFS 버전

In [ ]:
#n번정점을 업데이트 하려면 오일러투어 정점 리스트의 n번정점의 s~e까지 그 값으로 업데이트 해야한다.
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class LazySegmentTree:
  def __init__(self, data, default=0, func=max):
    """initialize the lazy segment tree with data"""
    self._default = default
    self._func = func

    self._len = len(data)
    self._size = _size = 1 << (self._len - 1).bit_length()
    self._lazy = [0] * (2 * _size)

    self.data = [default] * (2 * _size)
    self.data[_size:_size + self._len] = data
    for i in reversed(range(_size)):
      self.data[i] = func(self.data[i + i], self.data[i + i + 1])

  def _push(self, idx):
    """push query on idx to its children"""
    # Let the children know of the queries
    q, self._lazy[idx] = self._lazy[idx], 0

    self._lazy[2 * idx] += q
    self._lazy[2 * idx + 1] += q
    self.data[2 * idx] += q
    self.data[2 * idx + 1] += q

  def _update(self, idx):
    """updates the node idx to know of all queries applied to it via its ancestors"""
    for i in reversed(range(1, idx.bit_length())):
      self._push(idx >> i)

  def _build(self, idx):
    """make the changes to idx be known to its ancestors"""
    idx >>= 1
    while idx:
      self.data[idx] = self._func(self.data[2 * idx], self.data[2 * idx + 1]) + self._lazy[idx]
      idx >>= 1

  def add(self, start, stop, value):
    """lazily add value to [start, stop)"""
    start = start_copy = start + self._size
    stop = stop_copy = stop + self._size
    while start < stop:
      if start & 1:
        self._lazy[start] += value
        self.data[start] += value
        start += 1
      if stop & 1:
        stop -= 1
        self._lazy[stop] += value
        self.data[stop] += value
      start >>= 1
      stop >>= 1

    # Tell all nodes above of the updated area of the updates
    self._build(start_copy)
    self._build(stop_copy - 1)

  def query(self, start, stop, default=0):
    """func of data[start, stop)"""
    start += self._size
    stop += self._size

    # Apply all the lazily stored queries
    self._update(start)
    self._update(stop - 1)

    res = default
    while start < stop:
      if start & 1:
        res = self._func(res, self.data[start])
        start += 1
      if stop & 1:
        stop -= 1
        res = self._func(res, self.data[stop])
      start >>= 1
      stop >>= 1
    return res

def sol() :

  N, M = map(int, input().split())
  G = [[] for _ in range(N+1)]
  for i, p in enumerate(map(int, input().split())) :
    if p == -1 : continue
    G[p].append(i+1)
  
  S = [0] * (N+1) #i번 정점에 방문한 시점
  E = [0] * (N+1) #i번 정점을 빠져나온 시점
  cnt = 0

  ST = [(1, -1)]
  while ST:
    x, p = ST[-1]

    if S[x] == 0:
      cnt += 1
      S[x] = cnt

      for nx in G[x]:
        if nx == p: continue
        ST.append((nx, x))
    else:
      ST.pop()
      E[x] = cnt

  LST = LazySegmentTree([0] * (N+1), 0, lambda x, y : x+y)
  ans = []
  for _ in range(M) :
    q, *l = map(int, input().split())
    if q == 1 :
      x, w = l
      LST.add(S[x], E[x]+1, w)
    elif q == 2 :
      x = l[0]
      ans.append(LST.query(S[x], S[x]+1))

  sys.stdout.write('\n'.join(map(str, ans)))
  
sol()

- 동일한 문제로 14268(회사 문화 2)가 있다.

### 해석
- 트리가 그래프 입력으로 주워진다.
- 정점의 값이 증가/감소하면 그 아래에 있는 자식들의 값도 동일한 만큼, 재귀적으로 바뀐다.

### 풀이
- Euler Tour Traversal을 통해, 트리를 선형화한다(일렬로 편다)
  - 트리를 전위순회한 후 방문 순서에 따라 노드 번호를 다시 매길 수 있다.
  - 이렇게 되면 순회를 할 때 각 노드에 도달한 시점과, 그 노드 분기의 탐색을 끝내고 나가는 시점 사이에 방문한 노드들은, 그 노드를 루트로 하는 서브트리의 노드들이 되므로,\
  다시 매겨진 노드의 어떤 구간으로 표현할 수 있게 된다.
- 따라서 쿼리를 다음과 같이 정의할 수 있게 된다.
  - 루트를 `a`로 하는 서브트리에 대한 `x` 연산 `Q(a, x)` = 그 노드를 방문한 시점(`S`)과 떠나는 시점(`E`) 사이에 방문한 모든 노드들에 `x`를 연산 으로 변환할 수 있게 된다.